In [ ]:
# | default_exp _application.app

In [ ]:
# | export

import asyncio
import functools
import inspect
import json
import types
from asyncio import iscoroutinefunction  # do not use the version from inspect
from collections import namedtuple
from datetime import datetime, timedelta
from inspect import signature
from pathlib import Path
from typing import *
from unittest.mock import AsyncMock, MagicMock

import anyio
from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from fastcore.foundation import patch
from fastcore.meta import delegates
from pydantic import BaseModel

import fastkafka._components.logger

fastkafka._components.logger.should_supress_timestamps = True

import fastkafka
from fastkafka._components.aiokafka_consumer_loop import (
    aiokafka_consumer_loop,
    sanitize_kafka_config,
)
from fastkafka._components.aiokafka_producer_manager import AIOKafkaProducerManager
from fastkafka._components.asyncapi import (
    ConsumeCallable,
    ContactInfo,
    KafkaBroker,
    KafkaBrokers,
    KafkaServiceInfo,
    export_async_spec,
)
from fastkafka._components.helpers import filter_using_signature
from fastkafka._components.logger import get_logger
from fastkafka._components.producer_decorator import ProduceCallable, producer_decorator

In [ ]:
from fastkafka._components.logger import supress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
import os
import shutil
import unittest.mock

import pytest
import yaml
from pydantic import EmailStr, Field, HttpUrl

from fastkafka.testing import LocalKafkaBroker, mock_AIOKafkaProducer_send, true_after

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

### Constructor utilities

In [ ]:
# | export


@delegates(AIOKafkaConsumer)  # type: ignore
@delegates(AIOKafkaProducer, keep=True)  # type: ignore
def _get_kafka_config(
    **kwargs,
) -> Dict[str, Any]:
    """Get kafka config
    Args:
        bootstrap_servers (str, list(str)): a ``host[:port]`` string or list of
            ``host[:port]`` strings that the producer should contact to
            bootstrap initial cluster metadata. This does not have to be the
            full node list.  It just needs to have at least one broker that will
            respond to a Metadata API Request. Default port is 9092. If no
            servers are specified, will default to ``localhost:9092``.
        client_id (str): a name for this client. This string is passed in
            each request to servers and can be used to identify specific
            server-side log entries that correspond to this client.
            Default: ``aiokafka-producer-#`` (appended with a unique number
            per instance)
        key_serializer (Callable): used to convert user-supplied keys to bytes
            If not :data:`None`, called as ``f(key),`` should return
            :class:`bytes`.
            Default: :data:`None`.
        value_serializer (Callable): used to convert user-supplied message
            values to :class:`bytes`. If not :data:`None`, called as
            ``f(value)``, should return :class:`bytes`.
            Default: :data:`None`.
        acks (Any): one of ``0``, ``1``, ``all``. The number of acknowledgments
            the producer requires the leader to have received before considering a
            request complete. This controls the durability of records that are
            sent. The following settings are common:

            * ``0``: Producer will not wait for any acknowledgment from the server
              at all. The message will immediately be added to the socket
              buffer and considered sent. No guarantee can be made that the
              server has received the record in this case, and the retries
              configuration will not take effect (as the client won't
              generally know of any failures). The offset given back for each
              record will always be set to -1.
            * ``1``: The broker leader will write the record to its local log but
              will respond without awaiting full acknowledgement from all
              followers. In this case should the leader fail immediately
              after acknowledging the record but before the followers have
              replicated it then the record will be lost.
            * ``all``: The broker leader will wait for the full set of in-sync
              replicas to acknowledge the record. This guarantees that the
              record will not be lost as long as at least one in-sync replica
              remains alive. This is the strongest available guarantee.

            If unset, defaults to ``acks=1``. If `enable_idempotence` is
            :data:`True` defaults to ``acks=all``
        compression_type (str): The compression type for all data generated by
            the producer. Valid values are ``gzip``, ``snappy``, ``lz4``, ``zstd``
            or :data:`None`.
            Compression is of full batches of data, so the efficacy of batching
            will also impact the compression ratio (more batching means better
            compression). Default: :data:`None`.
        max_batch_size (int): Maximum size of buffered data per partition.
            After this amount :meth:`send` coroutine will block until batch is
            drained.
            Default: 16384
        linger_ms (int): The producer groups together any records that arrive
            in between request transmissions into a single batched request.
            Normally this occurs only under load when records arrive faster
            than they can be sent out. However in some circumstances the client
            may want to reduce the number of requests even under moderate load.
            This setting accomplishes this by adding a small amount of
            artificial delay; that is, if first request is processed faster,
            than `linger_ms`, producer will wait ``linger_ms - process_time``.
            Default: 0 (i.e. no delay).
        partitioner (Callable): Callable used to determine which partition
            each message is assigned to. Called (after key serialization):
            ``partitioner(key_bytes, all_partitions, available_partitions)``.
            The default partitioner implementation hashes each non-None key
            using the same murmur2 algorithm as the Java client so that
            messages with the same key are assigned to the same partition.
            When a key is :data:`None`, the message is delivered to a random partition
            (filtered to partitions with available leaders only, if possible).
        max_request_size (int): The maximum size of a request. This is also
            effectively a cap on the maximum record size. Note that the server
            has its own cap on record size which may be different from this.
            This setting will limit the number of record batches the producer
            will send in a single request to avoid sending huge requests.
            Default: 1048576.
        metadata_max_age_ms (int): The period of time in milliseconds after
            which we force a refresh of metadata even if we haven't seen any
            partition leadership changes to proactively discover any new
            brokers or partitions. Default: 300000
        request_timeout_ms (int): Produce request timeout in milliseconds.
            As it's sent as part of
            :class:`~kafka.protocol.produce.ProduceRequest` (it's a blocking
            call), maximum waiting time can be up to ``2 *
            request_timeout_ms``.
            Default: 40000.
        retry_backoff_ms (int): Milliseconds to backoff when retrying on
            errors. Default: 100.
        api_version (str): specify which kafka API version to use.
            If set to ``auto``, will attempt to infer the broker version by
            probing various APIs. Default: ``auto``
        security_protocol (str): Protocol used to communicate with brokers.
            Valid values are: ``PLAINTEXT``, ``SSL``. Default: ``PLAINTEXT``.
            Default: ``PLAINTEXT``.
        ssl_context (ssl.SSLContext): pre-configured :class:`~ssl.SSLContext`
            for wrapping socket connections. Directly passed into asyncio's
            :meth:`~asyncio.loop.create_connection`. For more
            information see :ref:`ssl_auth`.
            Default: :data:`None`
        connections_max_idle_ms (int): Close idle connections after the number
            of milliseconds specified by this config. Specifying :data:`None` will
            disable idle checks. Default: 540000 (9 minutes).
        enable_idempotence (bool): When set to :data:`True`, the producer will
            ensure that exactly one copy of each message is written in the
            stream. If :data:`False`, producer retries due to broker failures,
            etc., may write duplicates of the retried message in the stream.
            Note that enabling idempotence acks to set to ``all``. If it is not
            explicitly set by the user it will be chosen. If incompatible
            values are set, a :exc:`ValueError` will be thrown.
            New in version 0.5.0.
        sasl_mechanism (str): Authentication mechanism when security_protocol
            is configured for ``SASL_PLAINTEXT`` or ``SASL_SSL``. Valid values
            are: ``PLAIN``, ``GSSAPI``, ``SCRAM-SHA-256``, ``SCRAM-SHA-512``,
            ``OAUTHBEARER``.
            Default: ``PLAIN``
        sasl_plain_username (str): username for SASL ``PLAIN`` authentication.
            Default: :data:`None`
        sasl_plain_password (str): password for SASL ``PLAIN`` authentication.
            Default: :data:`None`
        sasl_oauth_token_provider (: class:`~aiokafka.abc.AbstractTokenProvider`):
            OAuthBearer token provider instance. (See
            :mod:`kafka.oauth.abstract`).
            Default: :data:`None`
        group_id (str or None): name of the consumer group to join for dynamic
            partition assignment (if enabled), and to use for fetching and
            committing offsets. If None, auto-partition assignment (via
            group coordinator) and offset commits are disabled.
            Default: None
        key_deserializer (Callable): Any callable that takes a
            raw message key and returns a deserialized key.
        value_deserializer (Callable, Optional): Any callable that takes a
            raw message value and returns a deserialized value.
        fetch_min_bytes (int): Minimum amount of data the server should
            return for a fetch request, otherwise wait up to
            `fetch_max_wait_ms` for more data to accumulate. Default: 1.
        fetch_max_bytes (int): The maximum amount of data the server should
            return for a fetch request. This is not an absolute maximum, if
            the first message in the first non-empty partition of the fetch
            is larger than this value, the message will still be returned
            to ensure that the consumer can make progress. NOTE: consumer
            performs fetches to multiple brokers in parallel so memory
            usage will depend on the number of brokers containing
            partitions for the topic.
            Supported Kafka version >= 0.10.1.0. Default: 52428800 (50 Mb).
        fetch_max_wait_ms (int): The maximum amount of time in milliseconds
            the server will block before answering the fetch request if
            there isn't sufficient data to immediately satisfy the
            requirement given by fetch_min_bytes. Default: 500.
        max_partition_fetch_bytes (int): The maximum amount of data
            per-partition the server will return. The maximum total memory
            used for a request ``= #partitions * max_partition_fetch_bytes``.
            This size must be at least as large as the maximum message size
            the server allows or else it is possible for the producer to
            send messages larger than the consumer can fetch. If that
            happens, the consumer can get stuck trying to fetch a large
            message on a certain partition. Default: 1048576.
        max_poll_records (int): The maximum number of records returned in a
            single call to :meth:`.getmany`. Defaults ``None``, no limit.
        auto_offset_reset (str): A policy for resetting offsets on
            :exc:`.OffsetOutOfRangeError` errors: ``earliest`` will move to the oldest
            available message, ``latest`` will move to the most recent, and
            ``none`` will raise an exception so you can handle this case.
            Default: ``latest``.
        enable_auto_commit (bool): If true the consumer's offset will be
            periodically committed in the background. Default: True.
        auto_commit_interval_ms (int): milliseconds between automatic
            offset commits, if enable_auto_commit is True. Default: 5000.
        check_crcs (bool): Automatically check the CRC32 of the records
            consumed. This ensures no on-the-wire or on-disk corruption to
            the messages occurred. This check adds some overhead, so it may
            be disabled in cases seeking extreme performance. Default: True
        partition_assignment_strategy (list): List of objects to use to
            distribute partition ownership amongst consumer instances when
            group management is used. This preference is implicit in the order
            of the strategies in the list. When assignment strategy changes:
            to support a change to the assignment strategy, new versions must
            enable support both for the old assignment strategy and the new
            one. The coordinator will choose the old assignment strategy until
            all members have been updated. Then it will choose the new
            strategy. Default: [:class:`.RoundRobinPartitionAssignor`]
        max_poll_interval_ms (int): Maximum allowed time between calls to
            consume messages (e.g., :meth:`.getmany`). If this interval
            is exceeded the consumer is considered failed and the group will
            rebalance in order to reassign the partitions to another consumer
            group member. If API methods block waiting for messages, that time
            does not count against this timeout. See `KIP-62`_ for more
            information. Default 300000
        rebalance_timeout_ms (int): The maximum time server will wait for this
            consumer to rejoin the group in a case of rebalance. In Java client
            this behaviour is bound to `max.poll.interval.ms` configuration,
            but as ``aiokafka`` will rejoin the group in the background, we
            decouple this setting to allow finer tuning by users that use
            :class:`.ConsumerRebalanceListener` to delay rebalacing. Defaults
            to ``session_timeout_ms``
        session_timeout_ms (int): Client group session and failure detection
            timeout. The consumer sends periodic heartbeats
            (`heartbeat.interval.ms`) to indicate its liveness to the broker.
            If no hearts are received by the broker for a group member within
            the session timeout, the broker will remove the consumer from the
            group and trigger a rebalance. The allowed range is configured with
            the **broker** configuration properties
            `group.min.session.timeout.ms` and `group.max.session.timeout.ms`.
            Default: 10000
        heartbeat_interval_ms (int): The expected time in milliseconds
            between heartbeats to the consumer coordinator when using
            Kafka's group management feature. Heartbeats are used to ensure
            that the consumer's session stays active and to facilitate
            rebalancing when new consumers join or leave the group. The
            value must be set lower than `session_timeout_ms`, but typically
            should be set no higher than 1/3 of that value. It can be
            adjusted even lower to control the expected time for normal
            rebalances. Default: 3000
        consumer_timeout_ms (int): maximum wait timeout for background fetching
            routine. Mostly defines how fast the system will see rebalance and
            request new data for new partitions. Default: 200
        exclude_internal_topics (bool): Whether records from internal topics
            (such as offsets) should be exposed to the consumer. If set to True
            the only way to receive records from an internal topic is
            subscribing to it. Requires 0.10+ Default: True
        isolation_level (str): Controls how to read messages written
            transactionally.

            If set to ``read_committed``, :meth:`.getmany` will only return
            transactional messages which have been committed.
            If set to ``read_uncommitted`` (the default), :meth:`.getmany` will
            return all messages, even transactional messages which have been
            aborted.

            Non-transactional messages will be returned unconditionally in
            either mode.

            Messages will always be returned in offset order. Hence, in
            `read_committed` mode, :meth:`.getmany` will only return
            messages up to the last stable offset (LSO), which is the one less
            than the offset of the first open transaction. In particular any
            messages appearing after messages belonging to ongoing transactions
            will be withheld until the relevant transaction has been completed.
            As a result, `read_committed` consumers will not be able to read up
            to the high watermark when there are in flight transactions.
            Further, when in `read_committed` the seek_to_end method will
            return the LSO. See method docs below. Default: ``read_uncommitted``
        sasl_oauth_token_provider (~aiokafka.abc.AbstractTokenProvider): OAuthBearer token provider instance. (See :mod:`kafka.oauth.abstract`).
            Default: None
    """
    allowed_keys = set(signature(_get_kafka_config).parameters.keys())
    if not set(kwargs.keys()) <= allowed_keys:
        unallowed_keys = ", ".join(
            sorted([f"'{x}'" for x in set(kwargs.keys()).difference(allowed_keys)])
        )
        raise ValueError(f"Unallowed key arguments passed: {unallowed_keys}")
    retval = kwargs.copy()

    # todo: check this values
    config_defaults = {
        "bootstrap_servers": "localhost:9092",
        "auto_offset_reset": "earliest",
        "max_poll_records": 100,
        #         "max_buffer_size": 10_000,
    }
    for key, value in config_defaults.items():
        if key not in retval:
            retval[key] = value

    return retval

In [ ]:
# print(combine_params(_get_kafka_config, AIOKafkaConsumer).__doc__)

In [ ]:
# signature(_get_kafka_config).parameters

In [ ]:
assert _get_kafka_config() == {
    "bootstrap_servers": "localhost:9092",
    "auto_offset_reset": "earliest",
    "max_poll_records": 100,
}

assert _get_kafka_config(max_poll_records=1_000) == {
    "bootstrap_servers": "localhost:9092",
    "auto_offset_reset": "earliest",
    "max_poll_records": 1_000,
}

In [ ]:
with pytest.raises(ValueError) as e:
    _get_kafka_config(random_key=1_000, whatever="whocares")
assert e.value.args == ("Unallowed key arguments passed: 'random_key', 'whatever'",)

In [ ]:
# | export


def _get_kafka_brokers(kafka_brokers: Optional[Dict[str, Any]] = None) -> KafkaBrokers:
    """Get Kafka brokers

    Args:
        kafka_brokers: Kafka brokers

    """
    if kafka_brokers is None:
        retval: KafkaBrokers = KafkaBrokers(
            brokers={
                "localhost": KafkaBroker(
                    url="https://localhost",
                    description="Local (dev) Kafka broker",
                    port="9092",
                )
            }
        )
    else:
        retval = KafkaBrokers(
            brokers={
                k: KafkaBroker.parse_raw(
                    v.json() if hasattr(v, "json") else json.dumps(v)
                )
                for k, v in kafka_brokers.items()
            }
        )

    return retval

In [ ]:
assert (
    _get_kafka_brokers(None).json()
    == '{"brokers": {"localhost": {"url": "https://localhost", "description": "Local (dev) Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}}}'
)

assert (
    _get_kafka_brokers(dict(localhost=dict(url="localhost"))).json()
    == '{"brokers": {"localhost": {"url": "localhost", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}}}'
)

assert (
    _get_kafka_brokers(
        dict(localhost=dict(url="localhost"), staging=dict(url="staging.airt.ai"))
    ).json()
    == '{"brokers": {"localhost": {"url": "localhost", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}, "staging": {"url": "staging.airt.ai", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}}}'
)

In [ ]:
# | export


def _get_topic_name(
    topic_callable: Union[ConsumeCallable, ProduceCallable], prefix: str = "on_"
) -> str:
    """Get topic name
    Args:
        topic_callable: a function
        prefix: prefix of the name of the function followed by the topic name

    Returns:
        The name of the topic
    """
    topic = topic_callable.__name__
    if not topic.startswith(prefix) or len(topic) <= len(prefix):
        raise ValueError(f"Function name '{topic}' must start with {prefix}")
    topic = topic[len(prefix) :]

    return topic

In [ ]:
def on_topic_name_1():
    pass


assert _get_topic_name(on_topic_name_1) == "topic_name_1"

assert _get_topic_name(on_topic_name_1, prefix="on_topic_") == "name_1"

In [ ]:
# | export


def _get_contact_info(
    name: str = "Author",
    url: str = "https://www.google.com",
    email: str = "noreply@gmail.com",
) -> ContactInfo:
    return ContactInfo(name=name, url=url, email=email)

In [ ]:
assert _get_contact_info() == ContactInfo(
    name="Author",
    url=HttpUrl(url="https://www.google.com", scheme="http"),
    email="noreply@gmail.com",
)

In [ ]:
# | export


class FastKafka:
    @delegates(_get_kafka_config)  # type: ignore
    def __init__(
        self,
        *,
        title: Optional[str] = None,
        description: Optional[str] = None,
        version: Optional[str] = None,
        contact: Optional[Dict[str, str]] = None,
        kafka_brokers: Optional[Dict[str, Any]] = None,
        root_path: Optional[Union[Path, str]] = None,
        **kwargs,
    ):
        """Creates FastKafka application

        Args:
            title: optional title for the documentation. If None,
                the title will be set to empty string
            description: optional description for the documentation. If
                None, the description will be set to empty string
            version: optional version for the documentation. If None,
                the version will be set to empty string
            contact: optional contact for the documentation. If None, the
                contact will be set to placeholder values:
                name='Author' url=HttpUrl('https://www.google.com', ) email='noreply@gmail.com'
            kafka_brokers: dictionary describing kafka brokers used for
                generating documentation
            root_path: path to where documentation will be created
            bootstrap_servers (str, list(str)): a ``host[:port]`` string or list of
                ``host[:port]`` strings that the producer should contact to
                bootstrap initial cluster metadata. This does not have to be the
                full node list.  It just needs to have at least one broker that will
                respond to a Metadata API Request. Default port is 9092. If no
                servers are specified, will default to ``localhost:9092``.
            client_id (str): a name for this client. This string is passed in
                each request to servers and can be used to identify specific
                server-side log entries that correspond to this client.
                Default: ``aiokafka-producer-#`` (appended with a unique number
                per instance)
            key_serializer (Callable): used to convert user-supplied keys to bytes
                If not :data:`None`, called as ``f(key),`` should return
                :class:`bytes`.
                Default: :data:`None`.
            value_serializer (Callable): used to convert user-supplied message
                values to :class:`bytes`. If not :data:`None`, called as
                ``f(value)``, should return :class:`bytes`.
                Default: :data:`None`.
            acks (Any): one of ``0``, ``1``, ``all``. The number of acknowledgments
                the producer requires the leader to have received before considering a
                request complete. This controls the durability of records that are
                sent. The following settings are common:

                * ``0``: Producer will not wait for any acknowledgment from the server
                  at all. The message will immediately be added to the socket
                  buffer and considered sent. No guarantee can be made that the
                  server has received the record in this case, and the retries
                  configuration will not take effect (as the client won't
                  generally know of any failures). The offset given back for each
                  record will always be set to -1.
                * ``1``: The broker leader will write the record to its local log but
                  will respond without awaiting full acknowledgement from all
                  followers. In this case should the leader fail immediately
                  after acknowledging the record but before the followers have
                  replicated it then the record will be lost.
                * ``all``: The broker leader will wait for the full set of in-sync
                  replicas to acknowledge the record. This guarantees that the
                  record will not be lost as long as at least one in-sync replica
                  remains alive. This is the strongest available guarantee.

                If unset, defaults to ``acks=1``. If `enable_idempotence` is
                :data:`True` defaults to ``acks=all``
            compression_type (str): The compression type for all data generated by
                the producer. Valid values are ``gzip``, ``snappy``, ``lz4``, ``zstd``
                or :data:`None`.
                Compression is of full batches of data, so the efficacy of batching
                will also impact the compression ratio (more batching means better
                compression). Default: :data:`None`.
            max_batch_size (int): Maximum size of buffered data per partition.
                After this amount :meth:`send` coroutine will block until batch is
                drained.
                Default: 16384
            linger_ms (int): The producer groups together any records that arrive
                in between request transmissions into a single batched request.
                Normally this occurs only under load when records arrive faster
                than they can be sent out. However in some circumstances the client
                may want to reduce the number of requests even under moderate load.
                This setting accomplishes this by adding a small amount of
                artificial delay; that is, if first request is processed faster,
                than `linger_ms`, producer will wait ``linger_ms - process_time``.
                Default: 0 (i.e. no delay).
            partitioner (Callable): Callable used to determine which partition
                each message is assigned to. Called (after key serialization):
                ``partitioner(key_bytes, all_partitions, available_partitions)``.
                The default partitioner implementation hashes each non-None key
                using the same murmur2 algorithm as the Java client so that
                messages with the same key are assigned to the same partition.
                When a key is :data:`None`, the message is delivered to a random partition
                (filtered to partitions with available leaders only, if possible).
            max_request_size (int): The maximum size of a request. This is also
                effectively a cap on the maximum record size. Note that the server
                has its own cap on record size which may be different from this.
                This setting will limit the number of record batches the producer
                will send in a single request to avoid sending huge requests.
                Default: 1048576.
            metadata_max_age_ms (int): The period of time in milliseconds after
                which we force a refresh of metadata even if we haven't seen any
                partition leadership changes to proactively discover any new
                brokers or partitions. Default: 300000
            request_timeout_ms (int): Produce request timeout in milliseconds.
                As it's sent as part of
                :class:`~kafka.protocol.produce.ProduceRequest` (it's a blocking
                call), maximum waiting time can be up to ``2 *
                request_timeout_ms``.
                Default: 40000.
            retry_backoff_ms (int): Milliseconds to backoff when retrying on
                errors. Default: 100.
            api_version (str): specify which kafka API version to use.
                If set to ``auto``, will attempt to infer the broker version by
                probing various APIs. Default: ``auto``
            security_protocol (str): Protocol used to communicate with brokers.
                Valid values are: ``PLAINTEXT``, ``SSL``. Default: ``PLAINTEXT``.
                Default: ``PLAINTEXT``.
            ssl_context (ssl.SSLContext): pre-configured :class:`~ssl.SSLContext`
                for wrapping socket connections. Directly passed into asyncio's
                :meth:`~asyncio.loop.create_connection`. For more
                information see :ref:`ssl_auth`.
                Default: :data:`None`
            connections_max_idle_ms (int): Close idle connections after the number
                of milliseconds specified by this config. Specifying :data:`None` will
                disable idle checks. Default: 540000 (9 minutes).
            enable_idempotence (bool): When set to :data:`True`, the producer will
                ensure that exactly one copy of each message is written in the
                stream. If :data:`False`, producer retries due to broker failures,
                etc., may write duplicates of the retried message in the stream.
                Note that enabling idempotence acks to set to ``all``. If it is not
                explicitly set by the user it will be chosen. If incompatible
                values are set, a :exc:`ValueError` will be thrown.
                New in version 0.5.0.
            sasl_mechanism (str): Authentication mechanism when security_protocol
                is configured for ``SASL_PLAINTEXT`` or ``SASL_SSL``. Valid values
                are: ``PLAIN``, ``GSSAPI``, ``SCRAM-SHA-256``, ``SCRAM-SHA-512``,
                ``OAUTHBEARER``.
                Default: ``PLAIN``
            sasl_plain_username (str): username for SASL ``PLAIN`` authentication.
                Default: :data:`None`
            sasl_plain_password (str): password for SASL ``PLAIN`` authentication.
                Default: :data:`None`
            sasl_oauth_token_provider (: class:`~aiokafka.abc.AbstractTokenProvider`):
                OAuthBearer token provider instance. (See
                :mod:`kafka.oauth.abstract`).
                Default: :data:`None`
            group_id (str or None): name of the consumer group to join for dynamic
                partition assignment (if enabled), and to use for fetching and
                committing offsets. If None, auto-partition assignment (via
                group coordinator) and offset commits are disabled.
                Default: None
            key_deserializer (Callable): Any callable that takes a
                raw message key and returns a deserialized key.
            value_deserializer (Callable, Optional): Any callable that takes a
                raw message value and returns a deserialized value.
            fetch_min_bytes (int): Minimum amount of data the server should
                return for a fetch request, otherwise wait up to
                `fetch_max_wait_ms` for more data to accumulate. Default: 1.
            fetch_max_bytes (int): The maximum amount of data the server should
                return for a fetch request. This is not an absolute maximum, if
                the first message in the first non-empty partition of the fetch
                is larger than this value, the message will still be returned
                to ensure that the consumer can make progress. NOTE: consumer
                performs fetches to multiple brokers in parallel so memory
                usage will depend on the number of brokers containing
                partitions for the topic.
                Supported Kafka version >= 0.10.1.0. Default: 52428800 (50 Mb).
            fetch_max_wait_ms (int): The maximum amount of time in milliseconds
                the server will block before answering the fetch request if
                there isn't sufficient data to immediately satisfy the
                requirement given by fetch_min_bytes. Default: 500.
            max_partition_fetch_bytes (int): The maximum amount of data
                per-partition the server will return. The maximum total memory
                used for a request ``= #partitions * max_partition_fetch_bytes``.
                This size must be at least as large as the maximum message size
                the server allows or else it is possible for the producer to
                send messages larger than the consumer can fetch. If that
                happens, the consumer can get stuck trying to fetch a large
                message on a certain partition. Default: 1048576.
            max_poll_records (int): The maximum number of records returned in a
                single call to :meth:`.getmany`. Defaults ``None``, no limit.
            auto_offset_reset (str): A policy for resetting offsets on
                :exc:`.OffsetOutOfRangeError` errors: ``earliest`` will move to the oldest
                available message, ``latest`` will move to the most recent, and
                ``none`` will raise an exception so you can handle this case.
                Default: ``latest``.
            enable_auto_commit (bool): If true the consumer's offset will be
                periodically committed in the background. Default: True.
            auto_commit_interval_ms (int): milliseconds between automatic
                offset commits, if enable_auto_commit is True. Default: 5000.
            check_crcs (bool): Automatically check the CRC32 of the records
                consumed. This ensures no on-the-wire or on-disk corruption to
                the messages occurred. This check adds some overhead, so it may
                be disabled in cases seeking extreme performance. Default: True
            partition_assignment_strategy (list): List of objects to use to
                distribute partition ownership amongst consumer instances when
                group management is used. This preference is implicit in the order
                of the strategies in the list. When assignment strategy changes:
                to support a change to the assignment strategy, new versions must
                enable support both for the old assignment strategy and the new
                one. The coordinator will choose the old assignment strategy until
                all members have been updated. Then it will choose the new
                strategy. Default: [:class:`.RoundRobinPartitionAssignor`]
            max_poll_interval_ms (int): Maximum allowed time between calls to
                consume messages (e.g., :meth:`.getmany`). If this interval
                is exceeded the consumer is considered failed and the group will
                rebalance in order to reassign the partitions to another consumer
                group member. If API methods block waiting for messages, that time
                does not count against this timeout. See `KIP-62`_ for more
                information. Default 300000
            rebalance_timeout_ms (int): The maximum time server will wait for this
                consumer to rejoin the group in a case of rebalance. In Java client
                this behaviour is bound to `max.poll.interval.ms` configuration,
                but as ``aiokafka`` will rejoin the group in the background, we
                decouple this setting to allow finer tuning by users that use
                :class:`.ConsumerRebalanceListener` to delay rebalacing. Defaults
                to ``session_timeout_ms``
            session_timeout_ms (int): Client group session and failure detection
                timeout. The consumer sends periodic heartbeats
                (`heartbeat.interval.ms`) to indicate its liveness to the broker.
                If no hearts are received by the broker for a group member within
                the session timeout, the broker will remove the consumer from the
                group and trigger a rebalance. The allowed range is configured with
                the **broker** configuration properties
                `group.min.session.timeout.ms` and `group.max.session.timeout.ms`.
                Default: 10000
            heartbeat_interval_ms (int): The expected time in milliseconds
                between heartbeats to the consumer coordinator when using
                Kafka's group management feature. Heartbeats are used to ensure
                that the consumer's session stays active and to facilitate
                rebalancing when new consumers join or leave the group. The
                value must be set lower than `session_timeout_ms`, but typically
                should be set no higher than 1/3 of that value. It can be
                adjusted even lower to control the expected time for normal
                rebalances. Default: 3000
            consumer_timeout_ms (int): maximum wait timeout for background fetching
                routine. Mostly defines how fast the system will see rebalance and
                request new data for new partitions. Default: 200
            exclude_internal_topics (bool): Whether records from internal topics
                (such as offsets) should be exposed to the consumer. If set to True
                the only way to receive records from an internal topic is
                subscribing to it. Requires 0.10+ Default: True
            isolation_level (str): Controls how to read messages written
                transactionally.

                If set to ``read_committed``, :meth:`.getmany` will only return
                transactional messages which have been committed.
                If set to ``read_uncommitted`` (the default), :meth:`.getmany` will
                return all messages, even transactional messages which have been
                aborted.

                Non-transactional messages will be returned unconditionally in
                either mode.

                Messages will always be returned in offset order. Hence, in
                `read_committed` mode, :meth:`.getmany` will only return
                messages up to the last stable offset (LSO), which is the one less
                than the offset of the first open transaction. In particular any
                messages appearing after messages belonging to ongoing transactions
                will be withheld until the relevant transaction has been completed.
                As a result, `read_committed` consumers will not be able to read up
                to the high watermark when there are in flight transactions.
                Further, when in `read_committed` the seek_to_end method will
                return the LSO. See method docs below. Default: ``read_uncommitted``
            sasl_oauth_token_provider (~aiokafka.abc.AbstractTokenProvider): OAuthBearer token provider instance. (See :mod:`kafka.oauth.abstract`).
                Default: None
        """

        # this is needed for documentation generation
        self._title = title if title is not None else ""
        self._description = description if description is not None else ""
        self._version = version if version is not None else ""
        if contact is not None:
            self._contact_info = _get_contact_info(**contact)
        else:
            self._contact_info = _get_contact_info()

        self._kafka_service_info = KafkaServiceInfo(
            title=self._title,
            version=self._version,
            description=self._description,
            contact=self._contact_info,
        )
        self._kafka_brokers = _get_kafka_brokers(kafka_brokers)

        self._root_path = Path(".") if root_path is None else Path(root_path)

        self._asyncapi_path = self._root_path / "asyncapi"
        (self._asyncapi_path / "docs").mkdir(exist_ok=True, parents=True)
        (self._asyncapi_path / "spec").mkdir(exist_ok=True, parents=True)

        # this is used as default parameters for creating AIOProducer and AIOConsumer objects
        self._kafka_config = _get_kafka_config(**kwargs)

        #
        self._consumers_store: Dict[str, Tuple[ConsumeCallable, Dict[str, Any]]] = {}

        self._producers_store: Dict[  # type: ignore
            str, Tuple[ProduceCallable, AIOKafkaProducer, Dict[str, Any]]
        ] = {}

        self._producers_list: List[  # type: ignore
            Union[AIOKafkaProducer, AIOKafkaProducerManager]
        ] = []

        # background tasks
        self._scheduled_bg_tasks: List[Callable[..., Coroutine[Any, Any, Any]]] = []
        self._bg_task_group_generator: Optional[anyio.abc.TaskGroup] = None
        self._bg_tasks_group: Optional[anyio.abc.TaskGroup] = None

        # todo: use this for errrors
        self._on_error_topic: Optional[str] = None

        self._is_started: bool = False
        self._is_shutting_down: bool = False
        self._kafka_consumer_tasks: List[asyncio.Task[Any]] = []
        self._kafka_producer_tasks: List[asyncio.Task[Any]] = []
        self._running_bg_tasks: List[asyncio.Task[Any]] = []
        self.run = False

        # testing functions
        self.AppMocks = None
        self.mocks = None
        self.awaited_mocks = None

    @property
    def is_started(self) -> bool:
        return self._is_started

    def set_bootstrap_servers(self, bootstrap_servers: str) -> None:
        self._kafka_config["bootstrap_servers"] = bootstrap_servers

    async def __aenter__(self) -> "FastKafka":
        await self.startup()
        return self

    async def __aexit__(
        self,
        exc_type: Optional[Type[BaseException]],
        exc: Optional[BaseException],
        tb: Optional[types.TracebackType],
    ) -> None:
        await self.shutdown()

    async def startup(self) -> None:
        raise NotImplementedError

    async def shutdown(self) -> None:
        raise NotImplementedError

    def consumes(
        self,
        topic: Optional[str] = None,
        *,
        prefix: str = "on_",
        **kwargs: Dict[str, Any],
    ) -> ConsumeCallable:
        raise NotImplementedError

    def produces(  # type: ignore
        self,
        topic: Optional[str] = None,
        *,
        prefix: str = "to_",
        producer: Optional[AIOKafkaProducer] = None,
        **kwargs: Dict[str, Any],
    ) -> ProduceCallable:
        raise NotImplementedError

    def run_in_background(
        self,
    ) -> Callable[[], Any]:
        raise NotImplementedError

    def _populate_consumers(
        self,
        is_shutting_down_f: Callable[[], bool],
    ) -> None:
        raise NotImplementedError

    def get_topics(self) -> Iterable[str]:
        raise NotImplementedError

    async def _populate_producers(self) -> None:
        raise NotImplementedError

    async def _populate_bg_tasks(self) -> None:
        raise NotImplementedError

    def create_docs(self) -> None:
        raise NotImplementedError

    def create_mocks(self) -> None:
        raise NotImplementedError

    async def _shutdown_consumers(self) -> None:
        raise NotImplementedError

    async def _shutdown_producers(self) -> None:
        raise NotImplementedError

    async def _shutdown_bg_tasks(self) -> None:
        raise NotImplementedError

In [ ]:
# | export


FastKafka.__module__ = "fastkafka"

In [ ]:
kafka_app = FastKafka()
kafka_app.__dict__

{'_title': '',
 '_description': '',
 '_version': '',
 '_contact_info': ContactInfo(name='Author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com'),
 '_kafka_service_info': KafkaServiceInfo(title='', version='', description='', contact=ContactInfo(name='Author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')),
 '_kafka_brokers': KafkaBrokers(brokers={'localhost': KafkaBroker(url='https://localhost', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)}),
 '_root_path': PosixPath('.'),
 '_asyncapi_path': PosixPath('asyncapi'),
 '_kafka_config': {'bootstrap_servers': 'localhost:9092',
  'auto_offset_reset': 'earliest',
  'max_poll_records': 100},
 '_consumers_store': {},
 '_producers_store': {},
 '_producers_list': [],
 '_scheduled_bg_tasks': [],
 '_bg_task_group_generator': None,
 '_bg_tasks_group': None,
 '_on_error_topic': None,
 '_is_started': False,
 '_is_shutting_down': False,
 '_kafka_consumer_tasks': [],
 '_k

In [ ]:
kafka_app = FastKafka(contact={"name": "Davor"})
kafka_app.__dict__

{'_title': '',
 '_description': '',
 '_version': '',
 '_contact_info': ContactInfo(name='Davor', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com'),
 '_kafka_service_info': KafkaServiceInfo(title='', version='', description='', contact=ContactInfo(name='Davor', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')),
 '_kafka_brokers': KafkaBrokers(brokers={'localhost': KafkaBroker(url='https://localhost', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)}),
 '_root_path': PosixPath('.'),
 '_asyncapi_path': PosixPath('asyncapi'),
 '_kafka_config': {'bootstrap_servers': 'localhost:9092',
  'auto_offset_reset': 'earliest',
  'max_poll_records': 100},
 '_consumers_store': {},
 '_producers_store': {},
 '_producers_list': [],
 '_scheduled_bg_tasks': [],
 '_bg_task_group_generator': None,
 '_bg_tasks_group': None,
 '_on_error_topic': None,
 '_is_started': False,
 '_is_shutting_down': False,
 '_kafka_consumer_tasks': [],
 '_kaf

In [ ]:
def create_testing_app(
    *, root_path: str = "/tmp/000_FastKafka", bootstrap_servers: str = "localhost:9092"
):
    if Path(root_path).exists():
        shutil.rmtree(root_path)

    kafka_app = FastKafka(
        kafka_brokers={
            "local": {
                "url": "kafka",
                "name": "development",
                "description": "Local (dev) Kafka broker",
                "port": 9092,
            }
        },
        root_path=root_path,
        bootstrap_servers=bootstrap_servers,
    )

    return kafka_app

In [ ]:
app = create_testing_app()
assert Path("/tmp/000_FastKafka").exists()
app

<fastkafka.FastKafka>

In [ ]:
# | export


@patch  # type: ignore
@delegates(AIOKafkaConsumer)  # type: ignore
def consumes(
    self: FastKafka,
    topic: Optional[str] = None,
    *,
    prefix: str = "on_",
    **kwargs: Dict[str, Any],
) -> Callable[[ConsumeCallable], ConsumeCallable]:
    """Decorator registering the callback called when a message is received in a topic.

    This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

    Args:
        topic: Kafka topic that the consumer will subscribe to and execute the
            decorated function when it receives a message from the topic,
            default: None. If the topic is not specified, topic name will be
            inferred from the decorated function name by stripping the defined prefix
        prefix: Prefix stripped from the decorated function to define a topic name
            if the topic argument is not passed, default: "on_". If the decorated
            function name is not prefixed with the defined prefix and topic argument
            is not passed, then this method will throw ValueError
        bootstrap_servers (str, list(str)): a ``host[:port]`` string (or list of
            ``host[:port]`` strings) that the consumer should contact to bootstrap
            initial cluster metadata.

            This does not have to be the full node list.
            It just needs to have at least one broker that will respond to a
            Metadata API Request. Default port is 9092. If no servers are
            specified, will default to ``localhost:9092``.
        client_id (str): a name for this client. This string is passed in
            each request to servers and can be used to identify specific
            server-side log entries that correspond to this client. Also
            submitted to :class:`~.consumer.group_coordinator.GroupCoordinator`
            for logging with respect to consumer group administration. Default:
            ``aiokafka-{version}``
        group_id (str or None): name of the consumer group to join for dynamic
            partition assignment (if enabled), and to use for fetching and
            committing offsets. If None, auto-partition assignment (via
            group coordinator) and offset commits are disabled.
            Default: None
        key_deserializer (Callable): Any callable that takes a
            raw message key and returns a deserialized key.
        value_deserializer (Callable, Optional): Any callable that takes a
            raw message value and returns a deserialized value.
        fetch_min_bytes (int): Minimum amount of data the server should
            return for a fetch request, otherwise wait up to
            `fetch_max_wait_ms` for more data to accumulate. Default: 1.
        fetch_max_bytes (int): The maximum amount of data the server should
            return for a fetch request. This is not an absolute maximum, if
            the first message in the first non-empty partition of the fetch
            is larger than this value, the message will still be returned
            to ensure that the consumer can make progress. NOTE: consumer
            performs fetches to multiple brokers in parallel so memory
            usage will depend on the number of brokers containing
            partitions for the topic.
            Supported Kafka version >= 0.10.1.0. Default: 52428800 (50 Mb).
        fetch_max_wait_ms (int): The maximum amount of time in milliseconds
            the server will block before answering the fetch request if
            there isn't sufficient data to immediately satisfy the
            requirement given by fetch_min_bytes. Default: 500.
        max_partition_fetch_bytes (int): The maximum amount of data
            per-partition the server will return. The maximum total memory
            used for a request ``= #partitions * max_partition_fetch_bytes``.
            This size must be at least as large as the maximum message size
            the server allows or else it is possible for the producer to
            send messages larger than the consumer can fetch. If that
            happens, the consumer can get stuck trying to fetch a large
            message on a certain partition. Default: 1048576.
        max_poll_records (int): The maximum number of records returned in a
            single call to :meth:`.getmany`. Defaults ``None``, no limit.
        request_timeout_ms (int): Client request timeout in milliseconds.
            Default: 40000.
        retry_backoff_ms (int): Milliseconds to backoff when retrying on
            errors. Default: 100.
        auto_offset_reset (str): A policy for resetting offsets on
            :exc:`.OffsetOutOfRangeError` errors: ``earliest`` will move to the oldest
            available message, ``latest`` will move to the most recent, and
            ``none`` will raise an exception so you can handle this case.
            Default: ``latest``.
        enable_auto_commit (bool): If true the consumer's offset will be
            periodically committed in the background. Default: True.
        auto_commit_interval_ms (int): milliseconds between automatic
            offset commits, if enable_auto_commit is True. Default: 5000.
        check_crcs (bool): Automatically check the CRC32 of the records
            consumed. This ensures no on-the-wire or on-disk corruption to
            the messages occurred. This check adds some overhead, so it may
            be disabled in cases seeking extreme performance. Default: True
        metadata_max_age_ms (int): The period of time in milliseconds after
            which we force a refresh of metadata even if we haven't seen any
            partition leadership changes to proactively discover any new
            brokers or partitions. Default: 300000
        partition_assignment_strategy (list): List of objects to use to
            distribute partition ownership amongst consumer instances when
            group management is used. This preference is implicit in the order
            of the strategies in the list. When assignment strategy changes:
            to support a change to the assignment strategy, new versions must
            enable support both for the old assignment strategy and the new
            one. The coordinator will choose the old assignment strategy until
            all members have been updated. Then it will choose the new
            strategy. Default: [:class:`.RoundRobinPartitionAssignor`]
        max_poll_interval_ms (int): Maximum allowed time between calls to
            consume messages (e.g., :meth:`.getmany`). If this interval
            is exceeded the consumer is considered failed and the group will
            rebalance in order to reassign the partitions to another consumer
            group member. If API methods block waiting for messages, that time
            does not count against this timeout. See `KIP-62`_ for more
            information. Default 300000
        rebalance_timeout_ms (int): The maximum time server will wait for this
            consumer to rejoin the group in a case of rebalance. In Java client
            this behaviour is bound to `max.poll.interval.ms` configuration,
            but as ``aiokafka`` will rejoin the group in the background, we
            decouple this setting to allow finer tuning by users that use
            :class:`.ConsumerRebalanceListener` to delay rebalacing. Defaults
            to ``session_timeout_ms``
        session_timeout_ms (int): Client group session and failure detection
            timeout. The consumer sends periodic heartbeats
            (`heartbeat.interval.ms`) to indicate its liveness to the broker.
            If no hearts are received by the broker for a group member within
            the session timeout, the broker will remove the consumer from the
            group and trigger a rebalance. The allowed range is configured with
            the **broker** configuration properties
            `group.min.session.timeout.ms` and `group.max.session.timeout.ms`.
            Default: 10000
        heartbeat_interval_ms (int): The expected time in milliseconds
            between heartbeats to the consumer coordinator when using
            Kafka's group management feature. Heartbeats are used to ensure
            that the consumer's session stays active and to facilitate
            rebalancing when new consumers join or leave the group. The
            value must be set lower than `session_timeout_ms`, but typically
            should be set no higher than 1/3 of that value. It can be
            adjusted even lower to control the expected time for normal
            rebalances. Default: 3000
        consumer_timeout_ms (int): maximum wait timeout for background fetching
            routine. Mostly defines how fast the system will see rebalance and
            request new data for new partitions. Default: 200
        api_version (str): specify which kafka API version to use.
            :class:`AIOKafkaConsumer` supports Kafka API versions >=0.9 only.
            If set to ``auto``, will attempt to infer the broker version by
            probing various APIs. Default: ``auto``
        security_protocol (str): Protocol used to communicate with brokers.
            Valid values are: ``PLAINTEXT``, ``SSL``. Default: ``PLAINTEXT``.
        ssl_context (ssl.SSLContext): pre-configured :class:`~ssl.SSLContext`
            for wrapping socket connections. Directly passed into asyncio's
            :meth:`~asyncio.loop.create_connection`. For more information see
            :ref:`ssl_auth`. Default: None.
        exclude_internal_topics (bool): Whether records from internal topics
            (such as offsets) should be exposed to the consumer. If set to True
            the only way to receive records from an internal topic is
            subscribing to it. Requires 0.10+ Default: True
        connections_max_idle_ms (int): Close idle connections after the number
            of milliseconds specified by this config. Specifying `None` will
            disable idle checks. Default: 540000 (9 minutes).
        isolation_level (str): Controls how to read messages written
            transactionally.

            If set to ``read_committed``, :meth:`.getmany` will only return
            transactional messages which have been committed.
            If set to ``read_uncommitted`` (the default), :meth:`.getmany` will
            return all messages, even transactional messages which have been
            aborted.

            Non-transactional messages will be returned unconditionally in
            either mode.

            Messages will always be returned in offset order. Hence, in
            `read_committed` mode, :meth:`.getmany` will only return
            messages up to the last stable offset (LSO), which is the one less
            than the offset of the first open transaction. In particular any
            messages appearing after messages belonging to ongoing transactions
            will be withheld until the relevant transaction has been completed.
            As a result, `read_committed` consumers will not be able to read up
            to the high watermark when there are in flight transactions.
            Further, when in `read_committed` the seek_to_end method will
            return the LSO. See method docs below. Default: ``read_uncommitted``
        sasl_mechanism (str): Authentication mechanism when security_protocol
            is configured for ``SASL_PLAINTEXT`` or ``SASL_SSL``. Valid values are:
            ``PLAIN``, ``GSSAPI``, ``SCRAM-SHA-256``, ``SCRAM-SHA-512``,
            ``OAUTHBEARER``.
            Default: ``PLAIN``
        sasl_plain_username (str): username for SASL ``PLAIN`` authentication.
            Default: None
        sasl_plain_password (str): password for SASL ``PLAIN`` authentication.
            Default: None
        sasl_oauth_token_provider (~aiokafka.abc.AbstractTokenProvider): OAuthBearer token provider instance. (See :mod:`kafka.oauth.abstract`).
            Default: None

    Returns:
        A function returning the same function

    Throws:
        ValueError

    """

    def _decorator(
        on_topic: ConsumeCallable,
        topic: Optional[str] = topic,
        kwargs: Dict[str, Any] = kwargs,
    ) -> ConsumeCallable:
        topic_resolved: str = (
            _get_topic_name(topic_callable=on_topic, prefix=prefix)
            if topic is None
            else topic
        )

        self._consumers_store[topic_resolved] = (on_topic, kwargs)

        return on_topic

    return _decorator

In [ ]:
# signature(FastKafka.consumes).parameters

In [ ]:
# print(combine_params(FastKafka.consumes, AIOKafkaConsumer).__doc__)

In [ ]:
app = create_testing_app()


# Basic check
@app.consumes()
def on_my_topic_1(msg: BaseModel) -> None:
    pass


assert app._consumers_store["my_topic_1"] == (on_my_topic_1, {}), app._consumers_store


# Check topic setting
@app.consumes(topic="test_topic_2")
def some_func_name(msg: BaseModel) -> None:
    pass


assert app._consumers_store["test_topic_2"] == (
    some_func_name,
    {},
), app._consumers_store


# Check prefix change
@app.consumes(prefix="for_")
def for_test_topic_3(msg: BaseModel) -> None:
    pass


assert app._consumers_store["test_topic_3"] == (
    for_test_topic_3,
    {},
), app._consumers_store

# Check passing of kwargs
kwargs = {"arg1": "val1", "arg2": 2}


@app.consumes(topic="test_topic", **kwargs)
def for_test_kwargs(msg: BaseModel):
    pass


assert app._consumers_store["test_topic"] == (
    for_test_kwargs,
    kwargs,
), app._consumers_store

In [ ]:
# | export


@patch  # type: ignore
@delegates(AIOKafkaProducer)  # type: ignore
def produces(
    self: FastKafka,
    topic: Optional[str] = None,
    *,
    prefix: str = "to_",
    **kwargs: Dict[str, Any],
) -> Callable[[ProduceCallable], ProduceCallable]:
    """Decorator registering the callback called when delivery report for a produced message is received

    This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

    Args:
        topic: Kafka topic that the producer will send returned values from
            the decorated function to, default: None- If the topic is not
            specified, topic name will be inferred from the decorated function
            name by stripping the defined prefix.
        prefix: Prefix stripped from the decorated function to define a topic
            name if the topic argument is not passed, default: "to_". If the
            decorated function name is not prefixed with the defined prefix
            and topic argument is not passed, then this method will throw ValueError
        producer: optional AIOKafkaProducer object used to produce messages
        bootstrap_servers (str, list(str)): a ``host[:port]`` string or list of
            ``host[:port]`` strings that the producer should contact to
            bootstrap initial cluster metadata. This does not have to be the
            full node list.  It just needs to have at least one broker that will
            respond to a Metadata API Request. Default port is 9092. If no
            servers are specified, will default to ``localhost:9092``.
        client_id (str): a name for this client. This string is passed in
            each request to servers and can be used to identify specific
            server-side log entries that correspond to this client.
            Default: ``aiokafka-producer-#`` (appended with a unique number
            per instance)
        key_serializer (Callable): used to convert user-supplied keys to bytes
            If not :data:`None`, called as ``f(key),`` should return
            :class:`bytes`.
            Default: :data:`None`.
        value_serializer (Callable): used to convert user-supplied message
            values to :class:`bytes`. If not :data:`None`, called as
            ``f(value)``, should return :class:`bytes`.
            Default: :data:`None`.
        acks (Any): one of ``0``, ``1``, ``all``. The number of acknowledgments
            the producer requires the leader to have received before considering a
            request complete. This controls the durability of records that are
            sent. The following settings are common:

            * ``0``: Producer will not wait for any acknowledgment from the server
              at all. The message will immediately be added to the socket
              buffer and considered sent. No guarantee can be made that the
              server has received the record in this case, and the retries
              configuration will not take effect (as the client won't
              generally know of any failures). The offset given back for each
              record will always be set to -1.
            * ``1``: The broker leader will write the record to its local log but
              will respond without awaiting full acknowledgement from all
              followers. In this case should the leader fail immediately
              after acknowledging the record but before the followers have
              replicated it then the record will be lost.
            * ``all``: The broker leader will wait for the full set of in-sync
              replicas to acknowledge the record. This guarantees that the
              record will not be lost as long as at least one in-sync replica
              remains alive. This is the strongest available guarantee.

            If unset, defaults to ``acks=1``. If `enable_idempotence` is
            :data:`True` defaults to ``acks=all``
        compression_type (str): The compression type for all data generated by
            the producer. Valid values are ``gzip``, ``snappy``, ``lz4``, ``zstd``
            or :data:`None`.
            Compression is of full batches of data, so the efficacy of batching
            will also impact the compression ratio (more batching means better
            compression). Default: :data:`None`.
        max_batch_size (int): Maximum size of buffered data per partition.
            After this amount :meth:`send` coroutine will block until batch is
            drained.
            Default: 16384
        linger_ms (int): The producer groups together any records that arrive
            in between request transmissions into a single batched request.
            Normally this occurs only under load when records arrive faster
            than they can be sent out. However in some circumstances the client
            may want to reduce the number of requests even under moderate load.
            This setting accomplishes this by adding a small amount of
            artificial delay; that is, if first request is processed faster,
            than `linger_ms`, producer will wait ``linger_ms - process_time``.
            Default: 0 (i.e. no delay).
        partitioner (Callable): Callable used to determine which partition
            each message is assigned to. Called (after key serialization):
            ``partitioner(key_bytes, all_partitions, available_partitions)``.
            The default partitioner implementation hashes each non-None key
            using the same murmur2 algorithm as the Java client so that
            messages with the same key are assigned to the same partition.
            When a key is :data:`None`, the message is delivered to a random partition
            (filtered to partitions with available leaders only, if possible).
        max_request_size (int): The maximum size of a request. This is also
            effectively a cap on the maximum record size. Note that the server
            has its own cap on record size which may be different from this.
            This setting will limit the number of record batches the producer
            will send in a single request to avoid sending huge requests.
            Default: 1048576.
        metadata_max_age_ms (int): The period of time in milliseconds after
            which we force a refresh of metadata even if we haven't seen any
            partition leadership changes to proactively discover any new
            brokers or partitions. Default: 300000
        request_timeout_ms (int): Produce request timeout in milliseconds.
            As it's sent as part of
            :class:`~kafka.protocol.produce.ProduceRequest` (it's a blocking
            call), maximum waiting time can be up to ``2 *
            request_timeout_ms``.
            Default: 40000.
        retry_backoff_ms (int): Milliseconds to backoff when retrying on
            errors. Default: 100.
        api_version (str): specify which kafka API version to use.
            If set to ``auto``, will attempt to infer the broker version by
            probing various APIs. Default: ``auto``
        security_protocol (str): Protocol used to communicate with brokers.
            Valid values are: ``PLAINTEXT``, ``SSL``. Default: ``PLAINTEXT``.
            Default: ``PLAINTEXT``.
        ssl_context (ssl.SSLContext): pre-configured :class:`~ssl.SSLContext`
            for wrapping socket connections. Directly passed into asyncio's
            :meth:`~asyncio.loop.create_connection`. For more
            information see :ref:`ssl_auth`.
            Default: :data:`None`
        connections_max_idle_ms (int): Close idle connections after the number
            of milliseconds specified by this config. Specifying :data:`None` will
            disable idle checks. Default: 540000 (9 minutes).
        enable_idempotence (bool): When set to :data:`True`, the producer will
            ensure that exactly one copy of each message is written in the
            stream. If :data:`False`, producer retries due to broker failures,
            etc., may write duplicates of the retried message in the stream.
            Note that enabling idempotence acks to set to ``all``. If it is not
            explicitly set by the user it will be chosen. If incompatible
            values are set, a :exc:`ValueError` will be thrown.
            New in version 0.5.0.
        sasl_mechanism (str): Authentication mechanism when security_protocol
            is configured for ``SASL_PLAINTEXT`` or ``SASL_SSL``. Valid values
            are: ``PLAIN``, ``GSSAPI``, ``SCRAM-SHA-256``, ``SCRAM-SHA-512``,
            ``OAUTHBEARER``.
            Default: ``PLAIN``
        sasl_plain_username (str): username for SASL ``PLAIN`` authentication.
            Default: :data:`None`
        sasl_plain_password (str): password for SASL ``PLAIN`` authentication.
            Default: :data:`None`
        sasl_oauth_token_provider (: class:`~aiokafka.abc.AbstractTokenProvider`):
            OAuthBearer token provider instance. (See
            :mod:`kafka.oauth.abstract`).
            Default: :data:`None`

    Returns:
        A function returning the same function

    Raises:
        ValueError: when needed
    """

    def _decorator(
        on_topic: ProduceCallable,
        topic: Optional[str] = topic,
        kwargs: Dict[str, Any] = kwargs,
    ) -> ProduceCallable:
        topic_resolved: str = (
            _get_topic_name(topic_callable=on_topic, prefix=prefix)
            if topic is None
            else topic
        )

        self._producers_store[topic_resolved] = (on_topic, None, kwargs)
        return producer_decorator(self._producers_store, on_topic, topic_resolved)

    return _decorator

In [ ]:
# signature(FastKafka.produces).parameters

In [ ]:
# print(combine_params(FastKafka.produces, AIOKafkaProducer).__doc__)

In [ ]:
app = create_testing_app()


# Basic check
async def to_my_topic_1(msg: BaseModel) -> None:
    pass


# Must be done without sugar to keep the original function reference
check_func = to_my_topic_1
to_my_topic_1 = app.produces()(to_my_topic_1)

assert app._producers_store["my_topic_1"] == (
    check_func,
    None,
    {},
), f"{app._producers_store}, {to_my_topic_1}"


# Check topic setting
def some_func_name(msg: BaseModel) -> None:
    pass


check_func = some_func_name
some_func_name = app.produces(topic="test_topic_2")(some_func_name)

assert app._producers_store["test_topic_2"] == (
    check_func,
    None,
    {},
), app._producers_store


# Check prefix change
@app.produces(prefix="for_")
def for_test_topic_3(msg: BaseModel) -> None:
    pass


check_func = for_test_topic_3
some_func_name = app.produces(prefix="for_")(for_test_topic_3)

assert app._producers_store["test_topic_3"] == (
    check_func,
    None,
    {},
), app._producers_store

# Check passing of kwargs
kwargs = {"arg1": "val1", "arg2": 2}


async def for_test_kwargs(msg: BaseModel):
    pass


check_func = for_test_kwargs
for_test_kwargs = app.produces(topic="test_topic", **kwargs)(for_test_kwargs)

assert app._producers_store["test_topic"] == (
    check_func,
    None,
    kwargs,
), app._producers_store

producer_decorator: producer_store={'my_topic_1': (<function to_my_topic_1>, None, {})}
producer_decorator: producer_store={'my_topic_1': (<function to_my_topic_1>, None, {}), 'test_topic_2': (<function some_func_name>, None, {})}
producer_decorator: producer_store={'my_topic_1': (<function to_my_topic_1>, None, {}), 'test_topic_2': (<function some_func_name>, None, {}), 'test_topic_3': (<function for_test_topic_3>, None, {})}
producer_decorator: producer_store={'my_topic_1': (<function to_my_topic_1>, None, {}), 'test_topic_2': (<function some_func_name>, None, {}), 'test_topic_3': (<function for_test_topic_3>, None, {})}
producer_decorator: producer_store={'my_topic_1': (<function to_my_topic_1>, None, {}), 'test_topic_2': (<function some_func_name>, None, {}), 'test_topic_3': (<function for_test_topic_3>, None, {}), 'test_topic': (<function for_test_kwargs>, None, {'arg1': 'val1', 'arg2': 2})}


In [ ]:
# | export


@patch  # type: ignore
def get_topics(self: FastKafka) -> Iterable[str]:
    produce_topics = set(self._producers_store.keys())
    consume_topics = set(self._consumers_store.keys())
    return consume_topics.union(produce_topics)

In [ ]:
app = create_testing_app()


@app.produces()
def to_topic_1() -> BaseModel:
    pass


@app.consumes()
def on_topic_2(msg: BaseModel):
    pass


assert app.get_topics() == set(["topic_1", "topic_2"]), f"{app.get_topics()=}"

producer_decorator: producer_store={'topic_1': (<function to_topic_1>, None, {})}


In [ ]:
# | export


@patch  # type: ignore
def run_in_background(
    self: FastKafka,
) -> Callable[
    [Callable[..., Coroutine[Any, Any, Any]]], Callable[..., Coroutine[Any, Any, Any]]
]:
    """
    Decorator to schedule a task to be run in the background.

    This decorator is used to schedule a task to be run in the background when the app's `_on_startup` event is triggered.

    Returns:
        Callable[None, None]: A decorator function that takes a background task as an input and stores it to be run in the backround.
    """

    def _decorator(
        bg_task: Callable[..., Coroutine[Any, Any, Any]]
    ) -> Callable[..., Coroutine[Any, Any, Any]]:
        """
        Store the background task.

        Args:
            bg_task (Callable[[], None]): The background task to be run asynchronously.

        Returns:
            Callable[[], None]: Original background task.
        """
        self._scheduled_bg_tasks.append(bg_task)

        return bg_task

    return _decorator

In [ ]:
# Check if the background job is getting registered

app = create_testing_app()


@app.run_in_background()
async def async_background_job():
    """Async background job"""
    pass


assert app._scheduled_bg_tasks[0] == async_background_job, app._scheduled_bg_tasks[0]
assert app._scheduled_bg_tasks.__len__() == 1

In [ ]:
class MyInfo(BaseModel):
    mobile: str = Field(..., example="+385987654321")
    name: str = Field(..., example="James Bond")


class MyMsgUrl(BaseModel):
    info: MyInfo = Field(..., example=dict(mobile="+385987654321", name="James Bond"))
    url: HttpUrl = Field(..., example="https://sis.gov.uk/agents/007")


class MyMsgEmail(BaseModel):
    msg_url: MyMsgUrl = Field(
        ...,
        example=dict(
            info=dict(mobile="+385987654321", name="James Bond"),
            url="https://sis.gov.uk/agents/007",
        ),
    )
    email: EmailStr = Field(..., example="agent-007@sis.gov.uk")


def setup_testing_app(bootstrap_servers="localhost:9092"):
    app = create_testing_app(bootstrap_servers=bootstrap_servers)

    @app.consumes("my_topic_1")
    def on_my_topic_one(msg: MyMsgUrl) -> None:
        logger.debug(f"on_my_topic_one(msg={msg},)")

    @app.consumes()
    async def on_my_topic_2(msg: MyMsgEmail) -> None:
        logger.debug(f"on_my_topic_2(msg={msg},)")

    with pytest.raises(ValueError) as e:

        @app.consumes()
        def my_topic_3(msg: MyMsgEmail) -> None:
            raise NotImplemented

    @app.produces()
    def to_my_topic_3(url: str) -> MyMsgUrl:
        logger.debug(f"on_my_topic_3(msg={url}")
        return MyMsgUrl(info=MyInfo("+3851987654321", "Sean Connery"), url=url)

    @app.produces()
    async def to_my_topic_4(msg: MyMsgEmail) -> MyMsgEmail:
        logger.debug(f"on_my_topic_4(msg={msg}")
        return msg

    @app.produces()
    def to_my_topic_5(url: str) -> MyMsgUrl:
        logger.debug(f"on_my_topic_5(msg={url}")
        return MyMsgUrl(info=MyInfo("+3859123456789", "John Wayne"), url=url)

    @app.run_in_background()
    async def long_bg_job():
        logger.debug(f"long_bg_job()")
        await asyncio.sleep(100)

    return app

In [ ]:
app = setup_testing_app()

assert set(app._consumers_store.keys()) == set(["my_topic_1", "my_topic_2"])
assert set(app._producers_store.keys()) == set(
    ["my_topic_3", "my_topic_4", "my_topic_5"]
)

print(f"app._kafka_service_info={app._kafka_service_info}")
print(f"app._kafka_brokers={app._kafka_brokers}")

producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {})}
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {}), 'my_topic_4': (<function setup_testing_app.<locals>.to_my_topic_4>, None, {})}
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {}), 'my_topic_4': (<function setup_testing_app.<locals>.to_my_topic_4>, None, {}), 'my_topic_5': (<function setup_testing_app.<locals>.to_my_topic_5>, None, {})}
app._kafka_service_info=title='' version='' description='' contact=ContactInfo(name='Author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')
app._kafka_brokers=brokers={'local': KafkaBroker(url='kafka', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)}


In [ ]:
# | export


@patch  # type: ignore
def _populate_consumers(
    self: FastKafka,
    is_shutting_down_f: Callable[[], bool],
) -> None:
    default_config: Dict[str, Any] = filter_using_signature(
        AIOKafkaConsumer, **self._kafka_config
    )
    self._kafka_consumer_tasks = [
        asyncio.create_task(
            aiokafka_consumer_loop(
                topic=topic,
                callback=consumer,
                msg_type=signature(consumer).parameters["msg"].annotation,
                is_shutting_down_f=is_shutting_down_f,
                **{**default_config, **override_config},
            )
        )
        for topic, (consumer, override_config) in self._consumers_store.items()
    ]


@patch  # type: ignore
async def _shutdown_consumers(
    self: FastKafka,
) -> None:
    if self._kafka_consumer_tasks:
        await asyncio.wait(self._kafka_consumer_tasks)

In [ ]:
async with LocalKafkaBroker() as bootstrap_server:
    app = setup_testing_app(bootstrap_servers=bootstrap_server)
    app._populate_consumers(is_shutting_down_f=true_after(1))
    assert len(app._kafka_consumer_tasks) == 2

    await app._shutdown_consumers()

    assert all([t.done() for t in app._kafka_consumer_tasks])

[INFO] fastkafka._components.helpers: Java is already installed.
[INFO] fastkafka._components.helpers: But not exported to PATH, exporting...
[INFO] fastkafka._components.helpers: Kafka is installed.
[INFO] fastkafka._components.helpers: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {})}
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {}), 'my_topic_4': (<function setup_testing_app.<locals>.to_my_topic_4>, None, {})}
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {}), 'my_topic_4': (<function setup_testing_app.<locals>.to_my_topic_4>, None, {}), 'my_topi

In [ ]:
# | export


# TODO: Add passing of vars
async def _create_producer(  # type: ignore
    *,
    callback: ProduceCallable,
    default_config: Dict[str, Any],
    override_config: Dict[str, Any],
    producers_list: List[Union[AIOKafkaProducer, AIOKafkaProducerManager]],
) -> Union[AIOKafkaProducer, AIOKafkaProducerManager]:
    """Creates a producer

    Args:
        callback: A callback function that is called when the producer is ready.
        producer: An existing producer to use.
        default_config: A dictionary of default configuration values.
        override_config: A dictionary of configuration values to override.
        producers_list: A list of producers to add the new producer to.

    Returns:
        A producer.
    """

    config = {
        **filter_using_signature(AIOKafkaProducer, **default_config),
        **override_config,
    }
    producer = AIOKafkaProducer(**config)
    logger.info(
        f"_create_producer() : created producer using the config: '{sanitize_kafka_config(**config)}'"
    )

    if not iscoroutinefunction(callback):
        producer = AIOKafkaProducerManager(producer)

    await producer.start()

    producers_list.append(producer)

    return producer


@patch  # type: ignore
async def _populate_producers(self: FastKafka) -> None:
    """Populates the producers for the FastKafka instance.

    Args:
        self: The FastKafka instance.

    Returns:
        None.

    Raises:
        None.
    """
    default_config: Dict[str, Any] = self._kafka_config
    self._producers_list = []
    self._producers_store.update(
        {
            topic: (
                callback,
                await _create_producer(
                    callback=callback,
                    default_config=default_config,
                    override_config=override_config,
                    producers_list=self._producers_list,
                ),
                override_config,
            )
            for topic, (
                callback,
                _,
                override_config,
            ) in self._producers_store.items()
        }
    )


@patch  # type: ignore
async def _shutdown_producers(self: FastKafka) -> None:
    [await producer.stop() for producer in self._producers_list[::-1]]
    # Remove references to stale producers
    self._producers_list = []
    self._producers_store.update(
        {
            topic: (
                callback,
                None,
                override_config,
            )
            for topic, (
                callback,
                _,
                override_config,
            ) in self._producers_store.items()
        }
    )

In [ ]:
async with LocalKafkaBroker() as bootstrap_server:
    app = setup_testing_app(bootstrap_servers=bootstrap_server)
    print(app._producers_store)
    await app._populate_producers()
    print(app._producers_store)
    assert len(app._producers_list) == 3
    print(app._producers_list)
    await app._shutdown_producers()

    # One more time for reentrancy
    await app._populate_producers()
    assert len(app._producers_list) == 3
    print(app._producers_list)
    await app._shutdown_producers()

[INFO] fastkafka._components.helpers: Java is already installed.
[INFO] fastkafka._components.helpers: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {})}
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {}), 'my_topic_4': (<function setup_testing_app.<locals>.to_my_topic_4>, None, {})}
producer_decorator: producer_store={'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {}), 'my_topic_4': (<function setup_testing_app.<locals>.to_my_topic_4>, None, {}), 'my_topic_5': (<function setup_testing_app.<locals>.to_my_topic_5>, None, {})}
{'my_topic_3': (<function setup_testing_app.<locals>.to_my_topic_3>, None, {}), 'my

In [ ]:
# | export


@patch  # type: ignore
async def _populate_bg_tasks(
    self: FastKafka,
) -> None:
    self._running_bg_tasks = [
        asyncio.create_task(task()) for task in self._scheduled_bg_tasks
    ]


@patch  # type: ignore
async def _shutdown_bg_tasks(
    self: FastKafka,
) -> None:
    [task.cancel() for task in self._running_bg_tasks]

    for task in self._running_bg_tasks:
        try:
            await task
        except asyncio.CancelledError:
            pass

In [ ]:
async with LocalKafkaBroker() as bootstrap_server:
    app = setup_testing_app(bootstrap_servers=bootstrap_server)
    await app._populate_bg_tasks()
    assert len(app._scheduled_bg_tasks) == 1
    assert len(app._running_bg_tasks) == 1
    await app._shutdown_bg_tasks()

In [ ]:
# | export


@patch  # type: ignore
async def startup(self: FastKafka) -> None:
    def is_shutting_down_f(self: FastKafka = self) -> bool:
        return self._is_shutting_down

    #     self.create_docs()
    await self._populate_producers()
    self._populate_consumers(is_shutting_down_f)
    await self._populate_bg_tasks()

    self._is_started = True


@patch  # type: ignore
async def shutdown(self: FastKafka) -> None:
    self._is_shutting_down = True

    await self._shutdown_bg_tasks()
    await self._shutdown_consumers()
    await self._shutdown_producers()

    self._is_shutting_down = False
    self._is_started = False

In [ ]:
# Test app reentrancy

async with LocalKafkaBroker() as bootstrap_server:
    with mock_AIOKafkaProducer_send() as mock:
        app = create_testing_app(bootstrap_servers=bootstrap_server)

        @app.produces()
        async def to_my_test_topic(mobile: str, url: str) -> MyMsgUrl:
            msg = MyMsgUrl(info=dict(mobile=mobile, name="James Bond"), url=url)
            return msg

        try:
            await app.startup()
            await to_my_test_topic(mobile="+385912345678", url="https://www.vip.hr")
        finally:
            await app.shutdown()

        try:
            await app.startup()
            await to_my_test_topic(mobile="+385987654321", url="https://www.ht.hr")
        finally:
            await app.shutdown()

        mock.assert_has_calls(
            [
                unittest.mock.call(
                    "my_test_topic",
                    b'{"info": {"mobile": "+385912345678", "name": "James Bond"}, "url": "https://www.vip.hr"}',
                    key=None,
                ),
                unittest.mock.call(
                    "my_test_topic",
                    b'{"info": {"mobile": "+385987654321", "name": "James Bond"}, "url": "https://www.ht.hr"}',
                    key=None,
                ),
            ]
        )

In [ ]:
# mock up send method of AIOKafkaProducer
async with LocalKafkaBroker() as bootstrap_server:
    with mock_AIOKafkaProducer_send() as mock:
        app = create_testing_app(bootstrap_servers=bootstrap_server)

        @app.produces()
        async def to_my_test_topic(mobile: str, url: str) -> MyMsgUrl:
            msg = MyMsgUrl(info=dict(mobile=mobile, name="James Bond"), url=url)
            return msg

        @app.produces()
        def to_my_test_topic_2(mobile: str, url: str) -> MyMsgUrl:
            msg = MyMsgUrl(info=dict(mobile=mobile, name="James Bond"), url=url)
            return msg

        try:
            await app.startup()
            await to_my_test_topic(mobile="+385912345678", url="https://www.vip.hr")
            to_my_test_topic_2(mobile="+385987654321", url="https://www.ht.hr")
        finally:
            await app.shutdown()

        mock.assert_has_calls(
            [
                unittest.mock.call(
                    "my_test_topic",
                    b'{"info": {"mobile": "+385912345678", "name": "James Bond"}, "url": "https://www.vip.hr"}',
                    key=None,
                ),
                unittest.mock.call(
                    "my_test_topic_2",
                    b'{"info": {"mobile": "+385987654321", "name": "James Bond"}, "url": "https://www.ht.hr"}',
                    key=None,
                ),
            ]
        )

In [ ]:
async with LocalKafkaBroker() as bootstrap_server:
    app = create_testing_app(bootstrap_servers=bootstrap_server)
    fast_task = unittest.mock.Mock()
    long_task = unittest.mock.Mock()

    @app.run_in_background()
    async def bg_task():
        fast_task()
        await asyncio.sleep(100)
        long_task()

    fast_task_second = unittest.mock.Mock()
    long_task_second = unittest.mock.Mock()

    @app.run_in_background()
    async def bg_task_second():
        fast_task_second()
        await asyncio.sleep(100)
        long_task_second()

    try:
        await app.startup()
        await asyncio.sleep(5)
    finally:
        await app.shutdown()

    fast_task.assert_called()
    long_task.assert_not_called()

    fast_task_second.assert_called()
    long_task_second.assert_not_called()

print("ok")

## Documentation generation

In [ ]:
# | export


@patch  # type: ignore
def create_docs(self: FastKafka) -> None:
    export_async_spec(
        consumers={
            topic: callback for topic, (callback, _) in self._consumers_store.items()
        },
        producers={
            topic: callback for topic, (callback, _, _) in self._producers_store.items()
        },
        kafka_brokers=self._kafka_brokers,
        kafka_service_info=self._kafka_service_info,
        asyncapi_path=self._asyncapi_path,
    )

In [ ]:
expected = """asyncapi: 2.5.0
channels:
  my_topic_1:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_2:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_3:
    publish:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_4:
    publish:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_5:
    publish:
      message:
        $ref: '#/components/messages/MyMsgUrl'
components:
  messages:
    MyMsgEmail:
      payload:
        example:
          email: agent-007@sis.gov.uk
          msg_url:
            info:
              mobile: '+385987654321'
              name: James Bond
            url: https://sis.gov.uk/agents/007
        properties:
          email:
            example: agent-007@sis.gov.uk
            format: email
            title: Email
            type: string
          msg_url:
            allOf:
            - $ref: '#/components/messages/MyMsgUrl'
            example:
              info:
                mobile: '+385987654321'
                name: James Bond
              url: https://sis.gov.uk/agents/007
            title: Msg Url
        required:
        - msg_url
        - email
        title: MyMsgEmail
        type: object
    MyMsgUrl:
      payload:
        example:
          info:
            mobile: '+385987654321'
            name: James Bond
          url: https://sis.gov.uk/agents/007
        properties:
          info:
            allOf:
            - $ref: '#/components/schemas/MyInfo'
            example:
              mobile: '+385987654321'
              name: James Bond
            title: Info
          url:
            example: https://sis.gov.uk/agents/007
            format: uri
            maxLength: 2083
            minLength: 1
            title: Url
            type: string
        required:
        - info
        - url
        title: MyMsgUrl
        type: object
  schemas:
    MyInfo:
      payload:
        properties:
          mobile:
            example: '+385987654321'
            title: Mobile
            type: string
          name:
            example: James Bond
            title: Name
            type: string
        required:
        - mobile
        - name
        title: MyInfo
        type: object
  securitySchemes: {}
info:
  contact:
    email: noreply@gmail.com
    name: Author
    url: https://www.google.com
  description: ''
  title: ''
  version: ''
servers:
  local:
    description: Local (dev) Kafka broker
    protocol: kafka
    url: kafka
    variables:
      port:
        default: '9092'
"""

In [ ]:
d1, d2 = None, None

docs_path = Path("/tmp/000_FastKafka/asyncapi/spec/asyncapi.yml")
if docs_path.exists():
    os.remove(docs_path)


async def test_me():
    global d1
    global d2
    app = setup_testing_app(bootstrap_servers=bootstrap_server)
    app.create_docs()
    with open(docs_path) as specs:
        d1 = yaml.safe_load(specs)
        d2 = yaml.safe_load(expected)
        assert d1 == d2, f"{d1} != {d2}"


asyncio.run(test_me())
print("ok")

## App mocks

In [ ]:
# | export


class AwaitedMock:
    @staticmethod
    def _await_for(f: Callable[..., Any]) -> Callable[..., Any]:
        @delegates(f)  # type: ignore
        async def inner(*args, f=f, timeout: int = 60, **kwargs) -> Any:
            if inspect.iscoroutinefunction(f):
                return await asyncio.wait_for(f(*args, **kwargs), timeout=timeout)
            else:
                t0 = datetime.now()
                e: Optional[Exception] = None
                while True:
                    try:
                        return f(*args, **kwargs)
                    except Exception as _e:
                        await asyncio.sleep(1)
                        e = _e

                    if datetime.now() - t0 > timedelta(seconds=timeout):
                        break

                raise e

        return inner  # type: ignore

    def __init__(self, o: Any):
        self._o = o

        for name in o.__dir__():
            if not name.startswith("_"):
                f = getattr(o, name)
                if inspect.ismethod(f):
                    setattr(self, name, self._await_for(f))

In [ ]:
# | export


@patch  # type: ignore
def create_mocks(self: FastKafka) -> None:
    """Creates self.mocks as a named tuple mapping a new function obtained by calling the original functions and a mock"""
    app_methods = [f for f, _ in self._consumers_store.values()] + [
        f for f, _, _ in self._producers_store.values()
    ]
    self.AppMocks = namedtuple(  # type: ignore
        f"{self.__class__.__name__}Mocks", [f.__name__ for f in app_methods]
    )

    self.mocks = self.AppMocks(  # type: ignore
        **{
            f.__name__: AsyncMock() if inspect.iscoroutinefunction(f) else MagicMock()
            for f in app_methods
        }
    )

    self.awaited_mocks = self.AppMocks(  # type: ignore
        **{name: AwaitedMock(mock) for name, mock in self.mocks._asdict().items()}
    )

    def add_mock(
        f: Callable[..., Any], mock: Union[AsyncMock, MagicMock]
    ) -> Callable[..., Any]:
        """Add call to mock when calling function f"""

        @functools.wraps(f)
        async def async_inner(
            *args: Any, f: Callable[..., Any] = f, mock: AsyncMock = mock, **kwargs: Any
        ) -> Any:
            await mock(*args, **kwargs)
            return await f(*args, **kwargs)

        @functools.wraps(f)
        def sync_inner(
            *args: Any, f: Callable[..., Any] = f, mock: MagicMock = mock, **kwargs: Any
        ) -> Any:
            mock(*args, **kwargs)
            return f(*args, **kwargs)

        if inspect.iscoroutinefunction(f):
            return async_inner
        else:
            return sync_inner

    self._consumers_store.update(
        {
            name: (
                add_mock(f, getattr(self.mocks, f.__name__)),
                kwargs,
            )
            for name, (f, kwargs) in self._consumers_store.items()
        }
    )

    self._producers_store.update(
        {
            name: (
                add_mock(f, getattr(self.mocks, f.__name__)),
                producer,
                kwargs,
            )
            for name, (f, producer, kwargs) in self._producers_store.items()
        }
    )

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


app = FastKafka(bootstrap_servers="localhost:9092")

app = FastKafka(bootstrap_servers="localhost:9092")


@app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

In [ ]:
app.create_mocks()
app.mocks.on_preprocessed_signals.assert_not_awaited()
app.mocks.to_predictions.assert_not_awaited()
app.create_mocks()
app.mocks.on_preprocessed_signals.assert_not_awaited()
app.mocks.to_predictions.assert_not_awaited()

In [ ]:
with pytest.raises(AssertionError) as e:
    await app.awaited_mocks.on_preprocessed_signals.assert_called_with(123, timeout=2)

In [ ]:
app.create_mocks()
app.mocks.on_preprocessed_signals.assert_not_awaited()
await app.awaited_mocks.on_preprocessed_signals.assert_not_awaited(timeout=3)